In [18]:
# Initial imports
import pandas as pd
import sklearn.linear_model

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

model_data = ('/content/model_df.csv')

# not working with current code
#pred_data = ('/content/for_predictions.csv')

In [19]:
# Activate functions

# function to turn csv data to datafram
def clean_set(data):
  data_df = pd.read_csv(data)

  data_df['ZIP_CODE'] = data_df['ZIP_CODE'].apply(lambda x: f'{x:05}')

  # set ZIP_CODE as index
  data_df = data_df.set_index('ZIP_CODE')

  data_df['EV_TOTAL'] = data_df.iloc[:, -9:-7].sum(axis=1)

  data_df = data_df[['EV_LEVEL_1', 'EV_LEVEL_2', 'EV_FAST', 'EV_TOTAL',
               'NETWORK_TYPE_ChargePoint',	'NETWORK_TYPE_Non_Networked',	'NETWORK_TYPE_Other_Network',
               'TOTAL_HOUSEHOLDS',	'MEDIAN_INCOME',
               'REG_COUNTS']]
  
  return data_df


# function to get target and features
def target_feature(dataframe):
  y = dataframe.REG_COUNTS

  X = dataframe[['EV_LEVEL_1', 'EV_LEVEL_2', 'EV_FAST', 'EV_TOTAL',
                    'NETWORK_TYPE_ChargePoint',	'NETWORK_TYPE_Non_Networked',	'NETWORK_TYPE_Other_Network',
                    'TOTAL_HOUSEHOLDS',	'MEDIAN_INCOME']]
  
  return (X,y)

# function to generate predictions from dataset
def predict(in_scaler, out_scaler, orig_df, ml_model, data_X):
  data_scaled = in_scaler.transform(data_X)

  pred_scaled = ml_model.predict(data_scaled)

  pred = out_scaler.inverse_transform(pred_scaled)

  pred_df = pd.DataFrame(pred, index=data_X.index, columns=['REG_PREDICT'])

  pred_out_df = orig_df.merge(pred_df, how='left', on='ZIP_CODE')

  return pred_out_df

In [20]:
model_df = clean_set(model_data)

# not working with current code
# pred_df = clean_set(pred_data)

# Train-Test-Split

In [21]:
model_X, model_y = target_feature(model_df)

# not working with current code
# pred_X, pred_y = target_feature(pred_df)

In [22]:
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(model_X, model_y, test_size=0.33, random_state=42)

# Scaling

In [23]:
input_scaler = StandardScaler()
output_scaler = StandardScaler()

X_train_scaled = input_scaler.fit_transform(X_train)
X_test_scaled = input_scaler.transform(X_test)

y_train_scaled = output_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled = output_scaler.transform(y_test.values.reshape(-1,1))

# Linear Regression

In [24]:
# Train the model using the training sets
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

#Export Predictions

In [25]:
model_predictions_df = predict(input_scaler, output_scaler, model_df, model, model_X)

# export ev_ml_out_df for Tableau.
model_predictions_df.to_csv('pred_out_df.csv')

# User Interactive

In [104]:
test_df = model_df.copy()

select_feature = input("\n Enter a feature from your dataset: \n EV_LEVEL_1, EV_LEVEL_2, EV_FAST, EV_TOTAL, \n NETWORK_TYPE_ChargePoint, NETWORK_TYPE_Non_Networked, NETWORK_TYPE_Other_Network, \n TOTAL_HOUSEHOLDS, MEDIAN_INCOME \n\n ")

modifier_input = input("Modify feature by what (whole number)? \n")

modifier = int(modifier_input)

# iterate over the dataframe row by row
for index_label, row_series in test_df.iterrows():
    # For each row update the 'Bonus' value to it's double
    test_df.at[index_label , select_feature] = row_series[select_feature] + modifier

pred_X, pred_y = target_feature(test_df)

feature_pred_df = predict(input_scaler, output_scaler, test_df, model, pred_X)

feature_pred_df['TARGET_CHANGE'] = feature_pred_df['REG_PREDICT'] - feature_pred_df['REG_COUNTS']

filename = (select_feature + "_" + str(modifier))

csv_name = "pred_%s.csv" % filename

feature_pred_df.to_csv(csv_name)

feature_pred_df


 Enter a feature from your dataset: 
 EV_LEVEL_1, EV_LEVEL_2, EV_FAST, EV_TOTAL, 
 NETWORK_TYPE_ChargePoint, NETWORK_TYPE_Non_Networked, NETWORK_TYPE_Other_Network, 
 TOTAL_HOUSEHOLDS, MEDIAN_INCOME 

 MEDIAN_INCOME
Modify feature by what (whole number)? 
5000


,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,EV_TOTAL,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non_Networked,NETWORK_TYPE_Other_Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME,REG_COUNTS,REG_PREDICT,TARGET_CHANGE
ZIP_CODE,,,,,,,,,,,,
03266,0,2,0,2,0,1,0,851.0,59904.0,0,-191.425168,-191.425168
07001,0,1,0,1,0,1,0,5022.0,74849.0,173,421.934393,248.934393
07003,0,12,5,12,4,1,3,18577.0,83034.0,800,2239.043385,1439.043385
07004,0,6,3,6,0,1,2,2621.0,127063.0,308,1652.676615,1344.676615
07005,0,1,10,1,0,1,1,5489.0,116946.0,446,1565.730968,1119.730968
...,...,...,...,...,...,...,...,...,...,...,...,...
99352,0,13,3,13,1,1,4,12877.0,86410.0,3413,1728.926816,-1684.073184
99354,36,48,0,84,0,45,1,9175.0,78369.0,2082,1102.605272,-979.394728
99361,0,1,0,1,0,0,1,750.0,74118.0,100,277.382568,177.382568
